In [9]:
import pickle
import numpy as np
import sympy as sym
from sympy.utilities.autowrap import autowrap
sym.init_printing()

In [10]:
def get_symbols():
    alpha, beta, gamma = sym.symbols('alpha beta gamma')  # spacecraft angles
    alphaDot, betaDot, gammaDot = sym.symbols('alphaDot betaDot gammaDot')  # spacecraft velocities
    theta = sym.symbols('theta1:8')  # joint angles assuming 7DoF
    thetaDot = sym.symbols('thetaDot1:8')  # joint velocities
    ang_s = (alpha, beta, gamma)
    omega_s = (alphaDot, betaDot, gammaDot)
    return theta, thetaDot, ang_s, omega_s

In [11]:
# states = Matrix([alpha, beta, gamma, theta_1, theta_2, theta_3, alpha_d, beta_d, gamma_d, theta_1d, theta_2d, theta_3d])
theta, thetaDot, ang_s, omega_s = get_symbols()
# MassMatrix = np.load('MassMatrix.npy', allow_pickle=True)
# Coriolis = np.load('CoriolisVector.npy', allow_pickle=True)

In [12]:
# with open('MassMat_sym.pickle', 'rb') as inM:
#     MassMatrix = pickle.loads(inM.read())

In [13]:
# Mass_C = autowrap(MassMatrix, backend='cython', tempdir='./MassMatrix')
# print('mass written')

In [14]:
# Mass_C = autowrap(MassMatrix, backend='cython', tempdir='./MassMatrix')
# print('mass written')
# Coriolis_C = autowrap(Coriolis, backend='cython', tempdir='./CoriolisVector')
# print('coriolis written')

In [15]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [19]:
%%writefile corio.pyxbld
import numpy as np
#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'Coriolis.c'],
                     include_dirs=['.', np.get_include()])

Overwriting corio.pyxbld


In [20]:
print('ok')

ok


In [21]:
%%cython_pyximport corio
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API
# here we just replicate the function signature from the header
cdef extern from "Coriolis.h":
    void Coriolis(double *y, double *dY)

# here is the "wrapper" signature that conforms to the odeint interface
def MassMatrix(cnp.ndarray[cnp.double_t, ndim=1] y, double t):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] dY = np.empty(14, dtype=np.double)
    # now call the C function
    Coriolis(<double *> y.data, <double *> dY.data)
    # return the result
    return dY

/home/ash/softwares/miniconda3/envs/codegen17/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/ash/Ash/repo/model_predictive_control/src/iros/corio.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


ImportError: Building module corio failed: ["distutils.errors.CompileError: command 'gcc' failed with exit status 4\n"]

In [ ]:
rnd = np.random.rand(10)
rnd
MassMatrix(rnd, 1)